### Face Detection


In [2]:
# Face Detection with smiles

import cv2
fd = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
sd = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_smile.xml")

# Video read using web cam
vid = cv2.VideoCapture(0)
counter = 0
captured = True
cropped_img = 0


# loop to read and show image until we break the loop
while captured :
    flag , img = vid.read()
    
    # if flag is true then only show image
    if flag:
        faces = fd.detectMultiScale(img,1.1 ,5)
        for x,y,w,h in faces:
            cropped_img = img[y:y+h, x:x+w]
            
            smiles = sd.detectMultiScale(cropped_img,1.1 ,5)
            for xs,ys,ws,hs in smiles:
                cv2.rectangle(img, pt1=(x+xs, y+ys), pt2 = (x+xs+ws, y+ys+hs), color=(0,0,255), thickness= 2)
            
            # Draw a rectange on img   
            cv2.rectangle(img, pt1=(x,y), pt2 = (x+w,y+h), color=(255,0,0), thickness= 2)
            
            # It will check if face is present in front of cam for 20 frames
            if len(faces) == 1:
                counter += 1
                print(counter)
                if counter == 20 :
                    captured = False
            else:
                counter = 0
            
        cv2.imshow("webcam_img",img)
        cv2.imshow("cropped_image",cropped_img)    
        key = cv2.waitKey(1)
        if key == ord("b"):
            break
    else:
        break
cv2.destroyAllWindows()
vid.release()

1
2
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


### Face Registration

In [1]:
# Face Registration

import cv2
import pandas as pd
import face_recognition as fr

fd = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Database for Faces
filename = "database.csv"
try:
    
    face_db = pd.read_csv(filename, index_col= 0)
    data = {"NAME":face_db["name"].values.tolist(),
            "ENCODING": face_db["enc"].values.tolist()}
except Exception as e:
    print(e)
    data = {"NAME":[], "ENCODING": []}

name = data["NAME"]
enc = data["ENCODING"]
print(name)
print(enc)

# Video read using web cam
vid = cv2.VideoCapture(0)
counter = 0
captured = True

# mai loop to read and show image until we break the loop
while captured :
    flag , img = vid.read()
    cropped_img = img
    
    # if flag is true then only show image
    if flag:
        faces = fd.detectMultiScale(img,1.1 ,5)
        for x,y,w,h in faces:
            cropped_img = img[y-50:y+h+50, x-50:x+w+50].copy()
            cv2.rectangle(img, pt1=(x,y), pt2= (x+w,y+h), color= (0,0,255), thickness=2)
            
            # It will check if face is present in front of cam for 20 frames
            if len(faces) == 1:
                counter += 1
                print(counter)
                if counter == 20:
                    name.append(input("Enter name:"))
                    face_enc = fr.face_encodings(cropped_img)
                    # print(face_enc)
                    enc.append(face_enc[0].tolist())
                    data = {"name":name, "enc": enc}
                    face_db = pd.DataFrame(data)
                    face_db.to_csv(filename, sep=",")
                    captured = False
            else:
                counter = 0
            
        cv2.imshow("cropped_image",cropped_img)
        key = cv2.waitKey(1)
        if key == ord("b"):
            break
    else:
        break
cv2.destroyAllWindows()
vid.release()

['bhupendra', 'mummy']
['[-0.10906095057725906, 0.19110354781150818, 0.08455847203731537, -0.058485135436058044, -0.14578299224376678, 0.052448440343141556, -0.017031047493219376, -0.09071125835180283, 0.264691025018692, -0.14030420780181885, 0.27552199363708496, 0.10803171992301941, -0.20994912087917328, -0.05921410769224167, 0.010685020126402378, 0.1472473293542862, -0.09111985564231873, -0.10861378908157349, -0.08192329108715057, -0.03515496850013733, 0.018785445019602776, 0.08360663056373596, 0.06092154607176781, 0.005063445772975683, -0.04268999397754669, -0.41908538341522217, -0.11840686202049255, -0.03309192508459091, 0.1370578557252884, -0.07693023234605789, 0.011817551217973232, -0.007245948538184166, -0.1415240317583084, -0.06404057890176773, 0.014679649844765663, 0.09624023735523224, -0.04817994683980942, -0.049115993082523346, 0.2118070274591446, 0.02917664125561714, -0.16564010083675385, -0.017340291291475296, 0.054678745567798615, 0.28107112646102905, 0.17848080396652222,

### Face Recognition

In [4]:
# Face Recognition

import cv2
import pandas as pd
import face_recognition as fr
import numpy as np

fd = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

filename = "database.csv"
try:
    face_db = pd.read_csv(filename, index_col= 0)
    data = {"NAME":face_db["name"].values.tolist(),
            "ENCODING": face_db["enc"].values.tolist()}
except Exception as e:
    print(e)
    data = {"NAME":[], "ENCODING": []}

name = data["NAME"]
enc = data["ENCODING"]
print(name)
print(enc)

# Video read using web cam
vid = cv2.VideoCapture(0)
counter = 0
captured = True

# loop to read and show image until we break the loop
while captured :
    flag , img = vid.read()
    cropped_img = img
    
    # if flag is true then only show image
    if flag:
        faces = fd.detectMultiScale(img,1.1 ,5)
        for x,y,w,h in faces:
            cropped_img = img[y:y+h, x:x+w].copy()
            
            # It will check if face is present in front of cam for 20 frames
            if len(faces) == 1:
                # Here the problem of index out of range is solved using try and except method
                try:
                    fresh_face_enc = fr.face_encodings(cropped_img)
                    for ind ,fe in enumerate(enc):
                        matched = fr.compare_faces(fresh_face_enc, np.array(eval(fe)))[0]
                        if matched:
                            print("Face Matched:",name[ind])
                            print()
                            cv2.putText(img, name[ind], (50,50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0,0,255), 4)
                            cv2.rectangle(img, pt1=(x,y), pt2= (x+w,y+h), color= (0,0,255), thickness=2)
                        else:
                            print("Face Not Matched")
                except IndexError:
                    print("No Face Detected")
            
        cv2.imshow("webcam_image",img)
        key = cv2.waitKey(1)
        if key == ord("b"):
            break
    else:
        break
cv2.destroyAllWindows()
vid.release()

['bhupendra', 'mummy']
['[-0.10906095057725906, 0.19110354781150818, 0.08455847203731537, -0.058485135436058044, -0.14578299224376678, 0.052448440343141556, -0.017031047493219376, -0.09071125835180283, 0.264691025018692, -0.14030420780181885, 0.27552199363708496, 0.10803171992301941, -0.20994912087917328, -0.05921410769224167, 0.010685020126402378, 0.1472473293542862, -0.09111985564231873, -0.10861378908157349, -0.08192329108715057, -0.03515496850013733, 0.018785445019602776, 0.08360663056373596, 0.06092154607176781, 0.005063445772975683, -0.04268999397754669, -0.41908538341522217, -0.11840686202049255, -0.03309192508459091, 0.1370578557252884, -0.07693023234605789, 0.011817551217973232, -0.007245948538184166, -0.1415240317583084, -0.06404057890176773, 0.014679649844765663, 0.09624023735523224, -0.04817994683980942, -0.049115993082523346, 0.2118070274591446, 0.02917664125561714, -0.16564010083675385, -0.017340291291475296, 0.054678745567798615, 0.28107112646102905, 0.17848080396652222,